In [1]:
%defaultDatasource jdbc:h2:file:./database

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
create view interacoes as
select D1.Name Name1, D2.Name Name2, count(*)  interacoes
from Drug D1, Drug D2, DrugUse DU1, DrugUse DU2
where D1.Code = DU1.CodeDrug and D2.Code = DU2.CodeDrug and DU1.IdPerson = DU2.IdPerson and D1.Code < D2.Code
group by Name1, Name2;

In [ ]:
SELECT * FROM interacoes;

In [ ]:
select I.Name1, I.Name2, I.interacoes
from interacoes I
where I.interacoes > 30
order by interacoes;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [4]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

In [5]:
create view maisTrinta as
select I.druga, I.drugb,I.weight
from DrugInteraction I
where I.weight > 30
order by I.weight desc;


In [6]:
select * from maisTrinta;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [2]:
create view total as
select sum(weight) totalRelacoes from DrugInteraction;
select * from total;

org.h2.jdbc.JdbcSQLException:  View "TOTAL" already exists; SQL statement

In [8]:
select I.druga, I.drugb, CAST(I.weight AS DOUBLE) / t.totalRelacoes suporte
from maisTrinta I, total t
order by suporte;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [3]:
create view totalTrinta2 as
select distinct I1.drugA, sum(I1.weight) totalDrug 
from Drug I1, DrugInteraction I2
where I1.code = I2.drugA or I1.code = I2.drugB
group by I1.drugA;

org.h2.jdbc.JdbcSQLException:  View "TOTALTRINTA2" already exists; SQL statement

In [10]:
select * from totalTrinta2;

In [4]:
select I.druga, I.drugb, CAST(I.weight AS DOUBLE) / t.totalDrug confianca
from maisTrinta I, totalTrinta2 t
where I.druga = t.druga
order by confianca;

In [5]:
select I.druga, I.drugb, CAST(I.weight AS DOUBLE) / t.totalDrug confianca
from maisTrinta I, totalTrinta2 t
where I.drugb = t.druga
order by confianca;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [2]:
create view drugColatheral2 as
select U.codeDrug, count(*) many
from DrugUse U, SideEffect S
where S.idPerson = U.idPerson
group by U.codeDrug;

In [3]:
select * from drugColatheral2;

In [7]:
create view drugColatheralConjunct2 as
select U.codeDrug DrugA, U2.CodeDrug DrugB,count(*) many
from DrugUse U, SideEffect S, DrugUse U2
where S.idPerson = U.idPerson and S.idPerson = U2.idPerson and U.codeDrug < U2.codeDrug
group by DrugA, DrugB;

In [2]:
select * from drugColatheralConjunct2;

In [5]:
create view drugDifference as
select U.DrugA, U.DrugB, sum(U.many) difference
from drugColatheralConjunct2 U, drugColatheralConjunct2 u2
where U.DrugA = U2.DrugA and U.DrugB != U2.DrugB
group by U.DrugA, U.DrugB;

In [ ]:
select C.drugA, C.drugB, CAST(C.many as DOUBLE)/D.difference proportion
from drugColatheralConjunct2 C, drugDifference D
where C.drugA = D.drugA and C.drugB = D.drugB;